#       **1.PRE-PROCESAMIENTO DE IMAGENES GEDI**

Permite el analisis de imagenes satelitales en la nube utilizando Google Earth Engine; se obtiene una imagen GEDI con datos de entrenamiento y validación para el modelo, suministrando información de Densidad de Biomasa aérea en unidades de Mg/ha. 

## 1.1 Importar librerias y cargar área de interés

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx
import shapely
import ee
import geemap

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-coberturap')

In [3]:
# --- File paths  ---
root_folder=r"/notebooks/ProyectoFinal/Shapes/"
adi_route = root_folder+'/PNNAmacayacu.shp'
adi = gpd.read_file(adi_route).to_crs(epsg=9377)
adi

,objectid_1,nombre,categoria,territoria,resolucion,hectareas_,escala,organizaci,fecha_act,url,...,centroid_y,fecha_regi,validado,hectarea_1,perimetro_,administra,id_pnn,record_id,app_id,geometry
0,198761,Amacayacu,Parque Nacional Natural,Territorial Amazonia,283,267480.32,None,Parques Nacionales Naturales de Colombia,2011-06-30,https://runap.parquesnacionales.gov.co/area-pr...,...,-3.439523,2011-06-30,Si,267240.603115,None,PNNC,27,2069,1,"POLYGON ((5331866.854 1221528.042, 5331899.919..."


## 1.2 Convertir geometria local a formato Earth Engine

In [4]:
ee_fc = geemap.gdf_to_ee(adi)     # Esto convierte todo el shapefile a una EE FeatureCollection
adi = ee_fc.geometry()

## 1.3 Llamar la colección GEDI04_MONTHLY de GEDI y seleccionar las bandas de interés

In [5]:
dataset = (ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY')
           .filterDate('2023-01-01', '2023-12-28')
           .filterBounds(adi))

info = dataset.getInfo()
print('Número de imágenes:', len(info['features']))
# Limitar a las primeras 10 imágenes para evitar sobrecargar Jupyter
coleccion_limitada = dataset.limit(20)

# Extraer los IDs de las imágenes
ids = coleccion_limitada.aggregate_array('system:id').getInfo()

# Imprimir los nombres (IDs)
print("Nombres de las imágenes:")
for i, nombre in enumerate(ids):
    print(f"{i+1}: {nombre}")


Número de imágenes: 3
Nombres de las imágenes:
1: LARSE/GEDI/GEDI04_A_002_MONTHLY/202301_072W_006S
2: LARSE/GEDI/GEDI04_A_002_MONTHLY/202302_072W_006S
3: LARSE/GEDI/GEDI04_A_002_MONTHLY/202303_072W_006S


## 1.4 Realizar la corrección de los datos de GEDI con máscaras de error de las mediciones

Se realiza también un mapa interactivo de visualización de la colección

In [6]:
##Function to select highest quality GEDI data
def quality_mask(image):
    return image.updateMask(image.select('l4_quality_flag').eq(1)) \
                .updateMask(image.select('degrade_flag').eq(0))

 
##Function to mask unreliable GEDI measurements
## with a relative standard error > 50% 
## agbd_se / agbd > 0.5
def errorMask(image):
  relative_se = image.select('agbd_se').divide(image.select('agbd'))
  return image.updateMask(relative_se.lte(0.6));


gediProjection = ee.Image(dataset.first()).select('agbd').projection()
gediProcessed = dataset.map(quality_mask).map(errorMask)
gediMosaic = gediProcessed.mosaic().select('agbd').setDefaultProjection(gediProjection);

##Definir los parámetros de visualización
agbdVis = {
  min: 0,
  max: 300,  ##Puedes ajustar según el rango de tus datos
  'palette': ['f7fcf5', 'c7e9c0', '74c476', '31a354', '006d2c', '00441b']
};

Map = geemap.Map()
Map.centerObject(adi)
Map.addLayer(gediMosaic, agbdVis, 'GEDI AGBD', True, 1)
Map.addLayer(adi, {}, 'PNN Amacayacu', 0)
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## 1.6. Exportar la composción de GEDI

In [7]:
# Export the composite image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=gediMosaic,
    description='GEDI_TrainingPoints',
    folder='earthengine',            # Destination folder in Google Drive
    fileNamePrefix='gedi', # File name without extension
    region=adi,                   # Area to export
    ##scale=25,                        # Set a very high limit to allow exporting large images
                                     # Earth Engine requires this to avoid exporting by mistake very large rasters
                                     # The default limit is 1e8 (100 million pixels), so we override it here

    maxPixels=1e13,                  # Allow large exports
    fileFormat='GeoTIFF'
)

# Start the export task
task.start()

print("Export task started. Monitor progress in the Earth Engine Tasks tab.")

Export task started. Monitor progress in the Earth Engine Tasks tab.
